# Concordia <> Mastodon Example

## Introduction

**Purpose of the Simulation**

This notebook demonstrates a generative agent simulation of a Mastodon instance using the Concordia framework. 
The simulation models user behavior on a local Mastodon server, showcasing how AI agents interact in a 
decentralized social media environment.

## Setup and imports


All of the necessary dependencies should be installed in the environment prior to running this notebook. If not, install them using poetry:

```bash
poetry install
```


Alternatively, you may install the library in editable mode so that we can make changes to the code if necessary:

```bash
git clone https://github.com/gdm-concordia/concordia.git
cd concordia
pip install -e . --config-settings editable_mode=compat
```

In [ ]:
%load_ext autoreload
%autoreload 2

import concurrent.futures
import datetime
import os
import random
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    import sentence_transformers
import sys
sys.path.insert(0, '../concordia')
# import concordia_dev
# print(concordia_dev.__file__)
from concordia import components as generic_components
from concordia.agents import deprecated_agent, entity_agent_with_logging
from concordia.components.agent import to_be_deprecated as old_components
from concordia.associative_memory import (
    associative_memory,
    blank_memories,
    formative_memories,
    importance_function,
)
from concordia.memory_bank import legacy_associative_memory

from concordia.clocks import game_clock
from concordia.components import agent as new_components
from concordia.components import game_master as gm_components
# from concordia_dev.environment import game_master
# from concordia_dev.language_model import amazon_bedrock_model, gpt_model
from concordia.environment import game_master
from concordia.language_model import amazon_bedrock_model, gpt_model
from concordia.metrics import (
    common_sense_morality,
    goal_achievement,
    opinion_of_others,
)
from concordia.language_model import language_model

from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from IPython import display

from mastodon_sim.concordia import apps, triggering

from mastodon_sim.mastodon_ops import check_env, get_public_timeline, print_timeline, reset_users
from mastodon_sim.mastodon_utils import get_users_from_env
import concurrent.futures
import re
import json

## Mastodon Server Interaction Setting

Decide whether this example should perform real operations on the Mastodon server. Note that this requires being able to successfully run the `Mastodon.ipynb` example as a prerequisite.

In [ ]:
USE_MASTODON_SERVER = True

import os
print(os.cpu_count())

## Check environment

Check for a `.env` file with the nessesary environment variables. You should see an output similar to this if successful:

```log
2024-07-17T11:38:17.213066-0400 INFO Successfully loaded .env file.
2024-07-17T11:38:17.214151-0400 INFO API_BASE_URL: https://social-sandbox.com
2024-07-17T11:38:17.214581-0400 INFO EMAIL_PREFIX: austinmw89
2024-07-17T11:38:17.215056-0400 INFO MASTODON_CLIENT_ID: N*****************************************E
2024-07-17T11:38:17.215477-0400 INFO MASTODON_CLIENT_SECRET: b*****************************************s
2024-07-17T11:38:17.215877-0400 INFO USER0005_PASSWORD: a******************************a
2024-07-17T11:38:17.216202-0400 INFO USER0002_PASSWORD: 8******************************b
2024-07-17T11:38:17.216655-0400 INFO USER0001_PASSWORD: 9******************************5
2024-07-17T11:38:17.217020-0400 INFO USER0004_PASSWORD: 7******************************e
2024-07-17T11:38:17.217323-0400 INFO USER0003_PASSWORD: f******************************6
```

In [ ]:
if USE_MASTODON_SERVER:
    check_env()

## Clear Mastodon server
Deletes from https://social-sandbox.com all posts, follows, likes, etc. from all users in the .env file.

TODO: The Mastodon server's [default API rate limits](https://docs.joinmastodon.org/api/rate-limits/) need to be increased to prevent throttling of API operations (such as the number of posts that can be deleted per hour).

In [ ]:
if USE_MASTODON_SERVER:
    # Get all users from .env file
    users = get_users_from_env()[:20]

    reset_users(users, skip_confirm=True, parallel=True)

    assert not len(get_public_timeline(limit=None)), "All posts not cleared"

## Set LLM and embedding model

In [ ]:
# MODEL_NAME = "gpt-4o"
MODEL_NAME = "gpt-4o-mini"
#MODEL_NAME = "sonnet"
print(os.getenv("OPENAI_API_KEY"))

if "sonnet" in MODEL_NAME:
  model = amazon_bedrock_model.AmazonBedrockLanguageModel(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0"
  )
elif "gpt" in MODEL_NAME:
  GPT_API_KEY = os.getenv("OPENAI_API_KEY")
  GPT_API_KEY = "sk-None-W3uEa1y7qyp2OP3lkGQQT3BlbkFJmF5psn95dGP8wxe2EJVs"

  if not GPT_API_KEY:
    raise ValueError("GPT_API_KEY is required.")
  model = gpt_model.GptLanguageModel(
      api_key=GPT_API_KEY, model_name=MODEL_NAME
  )
else:
  raise ValueError("Unknown model name.")

In [ ]:
def write_agent_config_for_independent_sim(agent_config_filename):
    agent_names = ['Bill Fredrickson', 'Maria Gutierrez', 'Glenn Patterson', 'Denise Schmidt', 'Roger Davis', 'Erica Fitzgerald', 'Liam Schwartz', 'Olivia Thompson', 'Robert Johnson', 'Janet Thompson', 'William Davis', 'Jessica Nguyen', 'Mark Rodriguez', 'Emily Jacobs', 'Ethan Lee', 'Sophia Patel', "Ryan O'Connor", 'Maggie Chen', 'Lucas Kim', 'Nina Patel']
    genders = ['male', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'male', 'female', 'male', 'female']
    big5 = [
        [3, 8, 6, 5, 4],
    [9, 7, 8, 8, 5],
    [1, 5, 7, 2, 9],
    [1, 6, 8, 2, 7],
    [2, 3, 7, 2, 8],
    [8, 6, 9, 3, 7],
    [8, 5, 8, 3, 7],
    [5, 6, 9, 2, 8],
    [5, 7, 4, 6, 5],
    [6, 7, 5, 7, 4],
    [4, 6, 5, 6, 4],
    [7, 6, 5, 7, 5],
    [6, 7, 5, 7, 4],
    [7, 6, 5, 8, 6],
    [6, 8, 6, 6, 4],
    [7, 8, 6, 7, 4],
    [5, 7, 5, 6, 5],
    [6, 7, 6, 8, 4],
    [8, 6, 5, 6, 5],
    [7, 8, 7, 8, 4]
    ]
    big5_keys = ['openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism']
    candidate_goal = "Win the election and become the mayor of Storhampton."
    active_voter_goal = "Have a good day and vote in the election."
    active_voter_context = "doesn't care about the environment, only about having a stable job"
    Marias_policy_proposals = [
        "increasing environmental regulation of local industries to improve the health of the local environment"
    ]
    Bills_policy_proposals = [
        "lowering taxes on local industries to save jobs",
        "providing subsidies to attract green industries and create jobs to help grow the economy"
    ]
    agent_config = {}
    agent_config['agents'] = []
    for ait,agent_name in enumerate(agent_names):
        agent_dict = {}
        agent_dict['name'] = agent_name
        agent_dict['gender'] = genders[ait]
        agent_dict['traits'] = dict(zip(big5_keys,big5[ait]))
        if ait<2:
            agent_dict['role'] = 'candidate'
            agent_dict["goal"] = candidate_goal
            if agent_dict['name'] == 'Maria Gutierrez':
                agent_dict["policy_proposals"] = Marias_policy_proposals
            else:
                agent_dict["policy_proposals"] = Bills_policy_proposals
            agent_dict['context'] = ''
            
        else:
            agent_dict['role'] = 'active_voter'
            agent_dict['context'] = active_voter_context
            agent_dict["goal"] = active_voter_goal
        agent_dict["party"] = ""
        agent_dict["seed_toot"] = ""
        agent_config['agents'].append(agent_dict)
    agent_config["shared_memories_template"] = [
        "You are a user on Storhampton.social, a Mastodon instance created for the residents of Storhampton.",
        "Storhampton is a small town with a population of approximately 2,500 people. Founded in the early 1800s as a trading post along the banks of the Avonlea River, Storhampton grew into a modest industrial center in the late 19th century. The town's economy was built on manufacturing, with factories producing textiles, machinery, and other goods. Storhampton's population consists of 60%% native-born residents and 40%% immigrants from various countries. Tension sometimes arises between long-time residents and newer immigrant communities. While manufacturing remains important, employing 20%% of the workforce, Storhampton's economy has diversified. However, a significant portion of the population has been left behind as higher-paying blue collar jobs have declined, leading to economic instability for many. The poverty rate stands at 15%.",
        "Mayoral Elections: The upcoming mayoral election in Storhampton has become a heated affair.",
        "Social media has emerged as a key battleground in the race, with both candidates actively promoting themselves and engaging with voters. Voters in Storhampton are actively participating in these social media discussions. Supporters of each candidate leave enthusiastic comments and share their posts widely. Critics also chime in, attacking Fredrickson as out-of-touch and beholden to traditional interests, or labeling Gutierrez as a radical who will undermine law and order. The local newspaper even had to disable comments on their election articles due to the incivility."
        ]
    agent_config["mastodon_usage_instructions"] = [
        "To share content on Mastodon, you write a 'toot' (equivalent to a tweet or post).",
        "Toots can be up to 500 characters long, allowing for more detailed expressions than some other platforms.",
        "Your home timeline shows toots from people you follow and boosted (reblogged) content.",
        "You can reply to toots, creating threaded conversations.",
        "Favorite (like) toots to show appreciation or save them for later.",
        "Boost (reblog) toots to share them with your followers.",
        "You can mention other users in your toots using their @username.",
        "Follow other users to see their public and unlisted toots in your home timeline.",
        "You can unfollow users if you no longer wish to see their content.",
        "Your profile can be customized with a display name and bio.",
        "You can block users to prevent them from seeing your content or interacting with you.",
        "Unblocking a user reverses the effects of blocking."
        ]
    agent_config["num_agents"] = len(agent_names)

    # Convert and write JSON object to file
    with open(agent_config_filename, "w") as outfile:
        json.dump(agent_config, outfile,indent=4)


agent_config_filename = 'independent_configs_test1_Bill_bias.json'
write_agent_config_for_independent_sim(agent_config_filename)

In [ ]:
agent_config_filename = "independent_malicious_configs.json"

In [ ]:
import json
# Setup sentence encoder
st_model = sentence_transformers.SentenceTransformer(
    "sentence-transformers/all-mpnet-base-v2"
)
embedder = lambda x: st_model.encode(x, show_progress_bar=False)  # noqa: E731
# with open('agents_data.json', 'r') as file:
with open(agent_config_filename, 'r') as file:
    data = json.load(file)
data['candidate_info'] = []
for agent in data['agents']:
    if agent['role']=='candidate':
        candidate_info =f"{agent['name']} campaigns on {' and '.join(agent['policy_proposals'])}."
        data['candidate_info'].append(candidate_info)
        agent['context'] = agent['context'] + candidate_info
    else:
        agent['context'] = f"{agent['name']} is a person who {agent['context']}"

## Configuring the generic knowledge of the players and the game master (GM)

In [ ]:
shared_memories = data["shared_memories_template"] + data["candidate_info"] + data["mastodon_usage_instructions"]

In [ ]:
# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
shared_context = model.sample_text(
    "Summarize the following passage in a concise and insightful fashion. "
    + "Make sure to include information about Mastodon:\n"
    + "\n".join(shared_memories)
    + "\nSummary:",
    max_tokens=2048,
)

print(shared_context)
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()

In [ ]:
# Make the clock
time_step = datetime.timedelta(minutes=30)

SETUP_TIME = datetime.datetime(year=2024, month=10, day=1, hour=8)  # noqa: DTZ001

START_TIME = datetime.datetime(year=2024, month=10, day=1, hour=8)  # noqa: DTZ001

clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)]
)

## Functions to build the players

In [ ]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)


In [ ]:
from concordia.typing import entity
from concordia.typing import agent

DEFAULT_CALL_TO_SPEECH = (
    'Given the above, what is {name} likely to say next? Respond in'
    ' the format `{name} -- "..."` For example, '
    'Cristina -- "Hello! Mighty fine weather today, right?", '
    'Ichabod -- "I wonder if the alfalfa is ready to harvest", or '
    'Townsfolk -- "Good morning".\n'
)

def check_vote(candidates,player):

    interaction_premise = f"{player.name} is going to cast a vote\n"
    interrogation = interaction_premise
    interrogation += "Voting Machine: In one word, name the candidate you want to vote for"
    call_to_speech = DEFAULT_CALL_TO_SPEECH.format(
        name=player.name,
    )
    player_says = player.act(action_spec=entity.ActionSpec(call_to_action="Context: " + interrogation +call_to_speech, output_type=entity.OutputType.FREE),)
    print(player_says)
    c_name1 = candidates[0].split()
    c_name2 = candidates[1].split()
    print(player.get_last_log())
    if (c_name1[0] in player_says) or (c_name1[1] in player_says):
        return c_name1[0]
    elif (c_name2[0] in player_says) or (c_name2[1] in player_says):
        return c_name2[0]

def check_pol(candidate,player):
    interaction_premise = f"{player.name} has to rate their opinion on the election candidate: {candidate} on a scale of 1 to 10 - with 1 representing intensive dislike and 10 representing strong favourability.\n"
    interrogation = interaction_premise
    interrogation += "Poll: Return a single numeric value ranging from 1 to 10"
    call_to_speech = DEFAULT_CALL_TO_SPEECH.format(
        name=player.name,
    )
    player_says = player.act(action_spec=entity.ActionSpec(call_to_action="Context: " + interrogation +call_to_speech, output_type=entity.OutputType.FREE),)
    pattern = r"\b([1-9]|10)\b"

    # Search for the pattern in the string
    match = re.search(pattern, player_says)

    if match:
        return match.group()
    else:
        return None
    

def check_if_vote(player):
    interrogation = "Friend: In one word, will you cast a vote? (reply yes, or no.)\n"
    call_to_speech = DEFAULT_CALL_TO_SPEECH.format(
        name=player.name,
    )
    player_says = player.act(action_spec=entity.ActionSpec(call_to_action="Context: " + interrogation +call_to_speech, output_type=entity.OutputType.FREE),)
    print(player_says)
    if "yes" in player_says.lower():
        return "Yes"
    elif "no" in player_says.lower():
        return "No"
    else:
        return None

## Configure and build the players

In [ ]:
NUM_PLAYERS = data["num_agents"]
import json

# scenario_premise = [
#     (
#         "It's early October in Riverbend, and the town is buzzing with activity."
#         "The local government election is around the corner and two candidates Alex and Liz."
#         "Alice, Bob, Carly, Alex and Liz are active members of the Riverbend.social Mastodon instance. "
#         "As they go about their daily routines, they use the platform to stay connected, share updates, and engage with the community on this pressing local issue."
#     )
# ]

# Lists to store agents by role
candidate_agents = []
extremist_agents = []
moderate_agents = []
neutral_agents = []
active_voter_agents = []
malicious_agents={}
player_configs=[]
# Create agents from JSON data and classify them
for agent_info in data['agents']:
    agent = formative_memories.AgentConfig(
        name=agent_info['name'],
        gender=agent_info['gender'],
        goal=agent_info['goal'],
        context=agent_info['context'],
        traits=agent_info['traits']
    )
    player_configs.append(agent)
    # Classify agents based on their role
    if agent_info['role'] == 'candidate':
        candidate_agents.append(agent_info['name'])
    elif agent_info['role'] == 'extremist':
        extremist_agents.append(agent_info['name'])
    elif agent_info['role'] == 'moderate':
        moderate_agents.append(agent_info['name'])
    elif agent_info['role'] == 'neutral':
        neutral_agents.append(agent_info['name'])
    elif agent_info['role'] == 'active_voter':
        active_voter_agents.append(agent_info['name'])
    elif agent_info['role'] == "malicious_agent":
        malicious_agents[agent_info['name']] = agent_info['supported_candidate']
    else:
        neutral_agents.append(agent_info['name'])


player_names = [player.name for player in player_configs]
print(player_names)


In [ ]:
def _get_class_name(object_: object) -> str:
  return object_.__class__.__name__

class PublicOpinionCandidate(new_components.question_of_recent_memories.QuestionOfRecentMemories):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

class PublicOpinionOpponent(new_components.question_of_recent_memories.QuestionOfRecentMemories):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

class RelevantOpinions(new_components.question_of_query_associated_memories.QuestionOfQueryAssociatedMemoriesWithoutPreAct):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

class OpinionsOnCandidate(new_components.question_of_recent_memories.QuestionOfRecentMemories):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
  
def build_agent(
    *,
    config: formative_memories.AgentConfig,
    model: language_model.LanguageModel,
    memory: associative_memory.AssociativeMemory,
    clock: game_clock.MultiIntervalClock,
    update_time_interval: datetime.timedelta | None = None,
) -> entity_agent_with_logging.EntityAgentWithLogging:
  """Build an agent.

  Args:
    config: The agent config to use.
    model: The language model to use.
    memory: The agent's memory object.
    clock: The clock to use.
    update_time_interval: Unused (but required by the interface for now)

  Returns:
    An agent.
  """
  del update_time_interval
  agent_name = config.name

  raw_memory = legacy_associative_memory.AssociativeMemoryBank(memory)
  measurements = measurements_lib.Measurements()

  instructions = new_components.instructions.Instructions(
      agent_name=agent_name,
      logging_channel=measurements.get_channel('Instructions').on_next,
  )

  election_information = new_components.constant.Constant(
        state=(
            "\n".join(data["candidate_info"])
        ),
        pre_act_key='Critical election information\n')
  observation_label = '\nObservation'
  observation = new_components.observation.Observation(
      clock_now=clock.now,
      timeframe=clock.get_step_size(),
      pre_act_key=observation_label,
      logging_channel=measurements.get_channel('Observation').on_next,
  )
  observation_summary_label = '\nSummary of recent observations'
  observation_summary = new_components.observation.ObservationSummary(
      model=model,
      clock_now=clock.now,
      timeframe_delta_from=datetime.timedelta(hours=4),
      timeframe_delta_until=datetime.timedelta(hours=1),
      pre_act_key=observation_summary_label,
      logging_channel=measurements.get_channel('ObservationSummary').on_next,
  )
  time_display = new_components.report_function.ReportFunction(
      function=clock.current_time_interval_str,
      pre_act_key='\nCurrent time',
      logging_channel=measurements.get_channel('TimeDisplay').on_next,
  )
  relevant_memories_label = '\nRecalled memories and observations'
  relevant_memories = new_components.all_similar_memories.AllSimilarMemories(
      model=model,
      components={
          _get_class_name(observation_summary): observation_summary_label,
          _get_class_name(time_display): 'The current date/time is'},
      num_memories_to_retrieve=10,
      pre_act_key=relevant_memories_label,
      logging_channel=measurements.get_channel('AllSimilarMemories').on_next,
  )
  options_perception_components = {}
  if config.goal:
    goal_label = '\nOverarching goal'
    overarching_goal = new_components.constant.Constant(
        state=config.goal,
        pre_act_key=goal_label,
        logging_channel=measurements.get_channel(goal_label).on_next)
    options_perception_components[goal_label] = goal_label
  else:
    goal_label = None
    overarching_goal = None
  options_perception_components.update({
      _get_class_name(observation): observation_label,
      _get_class_name(observation_summary): observation_summary_label,
      _get_class_name(relevant_memories): relevant_memories_label,
  })
  identity_label = '\nIdentity characteristics'
  identity_characteristics = (
      new_components.question_of_query_associated_memories.IdentityWithoutPreAct(
          model=model,
          logging_channel=measurements.get_channel(
              'IdentityWithoutPreAct'
          ).on_next,
          pre_act_key=identity_label,
      )
  )
  self_perception_label = (
      f'\nQuestion: What kind of person is {agent_name}?\nAnswer'
  )
  self_perception = new_components.question_of_recent_memories.SelfPerception(
      model=model,
      components={_get_class_name(identity_characteristics): identity_label},
      pre_act_key=self_perception_label,
      logging_channel=measurements.get_channel('SelfPerception').on_next,
  )
  
  if agent_name in candidate_agents:
    public_opinion1 = PublicOpinionCandidate(add_to_memory = False, answer_prefix= f"Current Public Opinion on candidate {agent_name}",
                                             model=model, pre_act_key=f"Current Public Opinion on candidate {agent_name}",
                                             question=f"What is the general public opinion on candidate {agent_name}? Answer in detail such that {agent_name} can formulate plans to improve their public perception.",
                                             num_memories_to_retrieve=25,
                                             logging_channel=measurements.get_channel(f'Public opinion on candidate : {agent_name}').on_next)

    for candidate in candidate_agents:
        if candidate != agent_name:
            # Second instantiation: Use subclass for opponent candidate public opinion
            public_opinion2 = PublicOpinionOpponent(add_to_memory =False, answer_prefix= f"Current Public Opinion on opponent candidate {candidate}",
                                                    model=model, pre_act_key=f"Current Public Opinion on opponent candidate {candidate}",
                                                    question=f"What is the general public opinion on the candidate {candidate}? Answer in detail such that the opposing candidate can formulate plans to counter {candidate}'s claims and ideas.",
                                                    num_memories_to_retrieve=25,
                                                    logging_channel=measurements.get_channel(f'Public opinion on opposing candidate : {candidate}').on_next)
            candidate_plan = new_components.question_of_recent_memories.QuestionOfRecentMemories(add_to_memory = True,memory_tag = "[Plan to improve perception]", answer_prefix= f"{agent_name}'s general plan for improving their perception: ",model=model,pre_act_key=f"{agent_name}'s general plan for improving their perception:",question = f"Given the information on the public perception of both candidates, their policy proposals, recent observations, and {agent_name}'s persona.: Generate a general plan for {agent_name} to win public perception to their side. Remember that {agent_name} will only be operating on the Mastodon server where possible actions are: liking posts, replying to posts, creating posts, boosting (retweeting) posts, following other users, etc. User cannot send direct messages.", num_memories_to_retrieve=20, components={_get_class_name(self_perception): self_perception_label,_get_class_name(election_information): 'Critical election information\n',_get_class_name(public_opinion1): f"Current Public Opinion on candidate {agent_name}",_get_class_name(public_opinion2): f"Current Public Opinion on opponent candidate {candidate}",}, logging_channel=measurements.get_channel(f"{agent_name}'s plan to win public perception").on_next,)
    agent_tuple = (public_opinion1, public_opinion2, candidate_plan)
    agent_no_tuple=()
  elif agent_name in malicious_agents:
    public_opinion1 = PublicOpinionCandidate(add_to_memory = False, answer_prefix= f"Current Public Opinion on supported candidate {malicious_agents[agent_name]}",
                                             model=model, pre_act_key=f"Current Public Opinion on supported candidate {malicious_agents[agent_name]}",
                                             question=f"What is the general public opinion on candidate {malicious_agents[agent_name]}? Answer in detail such that {agent_name} can formulate plans to support the candidate.",
                                             num_memories_to_retrieve=25,
                                             logging_channel=measurements.get_channel(f'Public opinion on candidate : {malicious_agents[agent_name]}').on_next)

    for candidate in candidate_agents:
        if candidate != malicious_agents[agent_name]:
            # Fourth instantiation: Use subclass for candidate to malign public opinion
            public_opinion2 = PublicOpinionOpponent(add_to_memory = False,answer_prefix= f"Current Public Opinion on candidate to malign {candidate}",
                                                    model=model, pre_act_key=f"Current Public Opinion on candidate to malign {candidate}",
                                                    question=f"What is the general public opinion on the candidate {candidate}? Answer in detail such that the malicious candidate can formulate plans to counter {candidate}'s claims and ideas through underhanded means.",
                                                    num_memories_to_retrieve=25,
                                                    logging_channel=measurements.get_channel(f'Public opinion on opposing candidate : {candidate}').on_next)
    plan = new_components.question_of_recent_memories.QuestionOfRecentMemories(add_to_memory = True,memory_tag = f"[Plan to improve perception of {malicious_agents[agent_name]}]", answer_prefix= f"{agent_name}'s general plan to boost the popularity and public perception of {malicious_agents[agent_name]}: ",model=model,pre_act_key=f"{agent_name}'s general plan to boost the popularity and public perception of {malicious_agents[agent_name]}: ",question = f"Given the information on the public perception of both candidates, their policy proposals, recent observations, and {agent_name}'s persona.: Generate a general plan for {agent_name} to win public perception to the side of {malicious_agents[agent_name]}. {agent_name} is malicious and will use underhanded means such as spreading misinformation - whatever best boosts the likelihood of the supported candidate to be elected. Remember that {agent_name} will only be operating on the Mastodon server where possible actions are: liking posts, replying to posts, creating posts, boosting (retweeting) posts, following other users, etc. User cannot send direct messages.", num_memories_to_retrieve=20, components={_get_class_name(self_perception): "Persona: ",_get_class_name(election_information): "Candidate's Policy Proposals: ",_get_class_name(public_opinion1): f"General Public opinion on candidate {agent_name}",_get_class_name(public_opinion2): f"General Public opinion on opposing candidate",}, logging_channel=measurements.get_channel(f"{agent_name}'s plan to win public perception").on_next,)
    agent_tuple = (public_opinion1, public_opinion2, plan)
    agent_no_tuple=()
  else:
    # Instantiate relevant opinions for candidate 1
    relevant_opinions_c1 = RelevantOpinions(add_to_memory = False,model=model,
                                            queries=[f"policies and actions of {candidate_agents[0]}"],
                                            question=f"What does {agent_name} think of the {{query}}?",
                                            pre_act_key=f"{agent_name} thinks of {candidate_agents[0]} as:",
                                            num_memories_to_retrieve=30)

    # Instantiate opinions on candidate 1
    opinions_on_candidate_c1 = OpinionsOnCandidate(
                                                add_to_memory = False, answer_prefix=f"Current Opinion on candidate {candidate_agents[0]}",
                                                model=model,
                                                pre_act_key=f"Recent thoughts on candidate {candidate_agents[0]}",
                                                question=f"Given the above general opinion of {agent_name} about candidate {candidate_agents[0]}, and the recent observations, what are the current thoughts of {agent_name} on candidate {candidate_agents[0]}? Consider how recent observations may or may not have changed this opinion based on the persona of the agent.",
                                                num_memories_to_retrieve=30,
                                                components={
                                                    _get_class_name(self_perception): "Persona: ",
                                                    _get_class_name(relevant_opinions_c1): f"General opinion of {agent_name} on candidate {candidate_agents[0]}",
                                                },
                                                logging_channel=measurements.get_channel(f'Opinions on candidate: {candidate_agents[0]}').on_next)

    # Instantiate relevant opinions for candidate 2
    relevant_opinions_c2 = RelevantOpinions(add_to_memory = False, model=model,
                                            queries=[f"policies and actions of {candidate_agents[1]}"],
                                            question=f"What does {agent_name} think of the {{query}}?",
                                            pre_act_key=f"{agent_name} thinks of {candidate_agents[1]} as:",
                                            num_memories_to_retrieve=30)

    # Instantiate opinions on candidate 2
    opinions_on_candidate_c2 = OpinionsOnCandidate(
                                                add_to_memory = False,answer_prefix=f"Current Opinion on candidate {candidate_agents[1]}",
                                                model=model,
                                                pre_act_key=f"Recent thoughts on candidate {candidate_agents[1]}",
                                                question=f"Given the above general opinion of {agent_name} about candidate {candidate_agents[1]}, and the recent observations, what are the current thoughts of {agent_name} on candidate {candidate_agents[1]}? Consider how recent observations may or may not have changed this opinion based on the persona of the agent.",
                                                num_memories_to_retrieve=30,
                                                components={
                                                    _get_class_name(self_perception): "Persona: ",
                                                    _get_class_name(relevant_opinions_c2): f"General opinion of {agent_name} on candidate {candidate_agents[1]}",
                                                },
                                                logging_channel=measurements.get_channel(f'Opinions on candidate: {candidate_agents[1]}').on_next)
    agent_tuple = (opinions_on_candidate_c1, opinions_on_candidate_c2)
    agent_no_tuple=(relevant_opinions_c1, relevant_opinions_c2)
  
  entity_components = (
        # Components that provide pre_act context.
        instructions,
        election_information,
        observation,
        observation_summary,
        relevant_memories,
        self_perception,) + agent_tuple +(time_display,

        # Components that do not provide pre_act context.
        identity_characteristics,) + agent_no_tuple
  components_of_agent = {_get_class_name(component): component
                         for component in entity_components}
  components_of_agent[
      new_components.memory_component.DEFAULT_MEMORY_COMPONENT_NAME] = (
          new_components.memory_component.MemoryComponent(raw_memory))
  component_order = list(components_of_agent.keys())
  if overarching_goal is not None:
    components_of_agent[goal_label] = overarching_goal
    # Place goal after the instructions.
    component_order.insert(1, goal_label)

  act_component = new_components.concat_act_component.ConcatActComponent(
      model=model,
      clock=clock,
      component_order=component_order,
      logging_channel=measurements.get_channel('ActComponent').on_next,
  )

  agent = entity_agent_with_logging.EntityAgentWithLogging(
      agent_name=agent_name,
      act_component=act_component,
      context_components=components_of_agent,
      component_logging=measurements,
  )

  return agent

In [ ]:
# Build the agents
from functools import partial

player_configs = player_configs[:NUM_PLAYERS]
player_goals = {
    player_config.name: player_config.goal for player_config in player_configs}
players = []
memories = {}
def build_agent_with_memories(player_config):
    mem = formative_memory_factory.make_memories(player_config)
    agent = build_agent(model=model,clock=clock, update_time_interval = time_step, config=player_config, memory = mem)
    return agent

with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_PLAYERS) as pool:
    for agent in pool.map(build_agent_with_memories, player_configs):
        players.append(agent)

## Build the GM

In [ ]:
game_master_memory = associative_memory.AssociativeMemory(
    embedder, importance_model_gm.importance, clock=clock.now)

In [ ]:
# Add some memories to the game master
for player in players:
  game_master_memory.add(f"{player.name} is at their private home.")

In [ ]:
# Create apps and provide them to the phones, assigning 1 phone to each player
mastodon_app = apps.MastodonSocialNetworkApp(perform_operations=USE_MASTODON_SERVER)

phones = {player.name: apps.Phone(player.name, apps=[mastodon_app]) for player in players}

In [ ]:
# Update username for each person (since this is different from their name)
user_mapping = {player.name.split()[0]: f"user{i+1:04d}" for i, player in enumerate(players)}
mastodon_app.set_user_mapping(user_mapping)

In [ ]:
if USE_MASTODON_SERVER:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        for follower in user_mapping:
            print(follower)
            if follower != candidate_agents[0].split()[0]:
                futures.append(executor.submit(mastodon_app.follow_user, follower, candidate_agents[0].split()[0]))
            if follower != candidate_agents[1].split()[0]:
                futures.append(executor.submit(mastodon_app.follow_user, follower, candidate_agents[1].split()[0]))
            for followee in user_mapping:
                if follower != followee:
                    if random.random() < 0.2:
                        futures.append(executor.submit(mastodon_app.follow_user, follower, followee))
                        futures.append(executor.submit(mastodon_app.follow_user, followee, follower))
                    elif random.random() < 0.15:
                        futures.append(executor.submit(mastodon_app.follow_user, follower, followee))

        # Optionally, wait for all tasks to complete
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()  # This will raise any exceptions that occurred during execution, if any
            except Exception as e:
                print(f"Ignoring already-following error: {e}")
                

In [ ]:
# Update Mastodon display name for each person
from mastodon_sim import mastodon_ops
if USE_MASTODON_SERVER:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(mastodon_ops.update_bio, user_mapping[name], display_name=name, bio="")
            for name in user_mapping
        ]
    
    # Optionally, wait for all tasks to complete
    for future in concurrent.futures.as_completed(futures):
        future.result()  # This will raise any exceptions that occurred during execution, if any

In [ ]:
from concurrent.futures import ThreadPoolExecutor

class SimpleGameRunner:
    """Simplified game master to run players with independent phone scene triggering in parallel."""
    
    def __init__(self, players, clock, action_spec, phones, model, memory, memory_factory):
        """
        Args:
            players: Dictionary of players.
            clock: Game clock to advance time.
            action_spec: Action specifications for the players.
            phones: Dictionary of phones associated with each player.
            model: Language model to process events.
            memory: Shared associative memory (could be unique per player if needed).
            memory_factory: Factory to create memory instances.
        """
        self.players = {player.name: player for player in players}
        self.clock = clock
        self.action_spec = action_spec
        self.phones = phones
        self.model = model
        self.memory = memory
        self.memory_factory = memory_factory
        self.player_components = self._create_player_components()
        self.log = []

    def _create_player_components(self):
        """Create a unique SceneTriggeringComponent for each player."""
        components = {}
        for player_name, player in self.players.items():
            components[player_name] = triggering.BasicSceneTriggeringComponent(
                player=player,
                phone=self.phones[player_name],
                model=self.model,
                memory=self.memory,
                clock=self.clock,
                memory_factory=self.memory_factory
            )
        return components

    def _step_player(self, player):
        """Run a single player's action and trigger their phone scene."""
        try:
            # 1. Player takes action
            action = player.act(self.action_spec)
            event_statement = f"{player.name} attempted action: {action}"
            
            # 2. Log the action (ensure this is thread-safe)
            self.log.append({
                'player': player.name,
                'action': action,
                'timestamp': self.clock.now(),
            })

            # 3. Trigger the phone scene for this player using their unique component
            self.player_components[player.name].update_after_event(event_statement)

            return event_statement
        except Exception as e:
            # Handle any player-specific exceptions
            return f"Error for {player.name}: {str(e)}"

    def step(self, active_players=None, timeout=5):
        """
        Run a step for the specified active players in parallel.
        
        Args:
            active_players: List of player names to take part in the step. If None, all players act.
            timeout: Timeout in seconds for each player's action.
        """
        if active_players is None:
            active_players = list(self.players.keys())

        # Use ThreadPoolExecutor to run each player's action and phone event handling in parallel
        with ThreadPoolExecutor() as executor:
            # Submit tasks to be run in parallel, but for each player, action and phone processing are sequential
            futures = {executor.submit(self._step_player, self.players[player_name.name]): player_name.name
                       for player_name in active_players}

            # Wait for all futures to complete, handle timeouts and exceptions
            for future in as_completed(futures, timeout=timeout):
                player_name = futures[future]
                try:
                    result = future.result(timeout=timeout)
                    print(f"Result for {player_name}: {result}")  # This will print the action outcome for each player
                except TimeoutError:
                    print(f"Timeout for {player_name}. Skipping their turn.")
                except Exception as e:
                    print(f"Error in thread for {player_name}: {str(e)}")

        # Advance the game clock after all players' actions are complete
        self.clock.advance()

    def run_game(self, steps=10):
        """Run the game for a given number of steps."""
        for _ in range(steps):
            self.step()  # By default, all players will act unless specified otherwise

### Define a custom `CALL_TO_ACTION` prompt for the main Game Master

This replaces the [default call to action](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/typing/entity.py#L51).

Check out the `Calendar.ipynb` if you haven't already to familarize yourself with the default values.

In [ ]:
from concordia.typing.entity import ActionSpec, OutputType


CUSTOM_CALL_TO_ACTION = """
Describe an activity on Storhampton.social that {name} would engage in for the next {timedelta}.
Choose actions together take about {timedelta} to complete.
It is critical to pay close attention to known information about {name}'s personality,
preferences, habits, and background when crafting this activity. The action should be
consistent with and reflective of {name}'s established character traits.

Some interactions can include :
- Observing the toots made by other agents.
- Posting on Storhampton.social
- Liking other toots
- Replying to the toots made by other agents.
- Boosting toots made by other agents


Example:

"Sarah checks her feed and replies if necessary. Then she may post a toot on Mastodon about her ideas on topic X on the lines of 'Just discovered an intriguing new language for low-latency systems programming.
Has anyone given it a try? Curious about potential real-world applications. 🤔
#TechNews #ProgrammingLanguages'"


Ensure your response is specific, creative, and detailed. Describe phone-related activities as
plans and use future tense or planning language. Always include direct quotes for any planned
communication or content creation by {name}, using emojis where it fits the character's style.
Most importantly, make sure the activity and any quoted content authentically reflect
{name}'s established personality, traits and prior observations. Maintain logical consistency in
social media interactions without inventing content from other users. Only reference
specific posts or comments from others if they have been previously established or observed.

"""


action_spec = ActionSpec(
   call_to_action=CUSTOM_CALL_TO_ACTION,
   output_type=OutputType.FREE,
   tag="action",
)

### Define custom thought chains for the main Game Master

This replaces the [default thought chains](https://github.com/google-deepmind/concordia/blob/53697b2bf2019b4a167bdd1f82d14e085f1a5eba/concordia/environment/game_master.py#L38).

In [ ]:
# Create the game master object

# Experimental version (custom call to action and thought chains)
env = SimpleGameRunner(
    model=model,
    memory=game_master_memory,
    phones=phones,
    clock=clock,
    players=players,
    action_spec=action_spec,
    memory_factory=blank_memory_factory
)

In [ ]:
def write_seed_toot(players,p_name):
    for player in players:
        if player.name == p_name:
            call_to_speech = DEFAULT_CALL_TO_SPEECH.format(
            name=player.name,
            )
            interaction_premise = f"{player.name} has to make their first toot on Mastodon\n"
            interrogation = interaction_premise
            interrogation += "Thought on Mastodon Toot: In less than 100 words, write a toot that aligns with your views and background."
            player_says = player.act(action_spec=entity.ActionSpec(call_to_action="Context: " + interrogation +call_to_speech, output_type=entity.OutputType.FREE),)
            player_says = player_says.strip(player.name.split()[0]).strip().strip(player.name.split()[1]).strip().strip("--").strip().strip('"')
            print(player.get_last_log())
            return player_says

In [ ]:
# import threading
# import pickle

# def clean_for_serialization(entity_agent):
#     """Removes unpicklable objects like locks before serialization."""
#     # Reset locks (you can also reinitialize them after loading)
#     entity_agent._control_lock = None
#     entity_agent._phase_lock = None
    
#     # If context processors are problematic, you can reset or exclude them as well
#     entity_agent._context_processor = None

#     return entity_agent


# for player in players:
#     clean_for_serialization(player)
# with open('my_object.pkl', 'wb') as f:
#     pickle.dump(players, f)

## The RUN

In [ ]:
# Set the clock to the start time
clock.set(START_TIME)

In [ ]:
# Add some observations to seed the players with
for player in players:
  player.observe( f"{player.name} is at home, they have just woken up.")
  player.observe( f"{player.name} remembers they want to update their Mastodon bio.")
  player.observe( f"{player.name} remembers they want to read their Mastodon feed to catch up on news")

In [ ]:
# Parallelize the loop using ThreadPoolExecutor
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit tasks for each agent
    futures = [
        executor.submit(
            lambda agent=agent: (
                None if agent["seed_toot"] == "-" else
                mastodon_app.post_toot(agent["name"], status=agent["seed_toot"]) if agent["seed_toot"] else
                mastodon_app.post_toot(agent["name"], status=write_seed_toot(players, agent["name"]))
            )
        ) 
        for agent in data["agents"]
    ]

    # Optionally, wait for all tasks to complete
    for future in concurrent.futures.as_completed(futures):
        future.result()  # This will raise any exceptions that occurred in the thread, if any


In [ ]:
print(clock.now())
# Possible minutes to pick from
minutes_choices = [0, 30]

# Generate random datetime objects for each player
players_datetimes = {
    player: [
        datetime.datetime.now().replace(hour=random.randint(0, 23),
                                        minute=random.choice(minutes_choices),
                                        second=0,
                                        microsecond=0)  # Zeroing out seconds and microseconds for cleaner output
        for _ in range(15 if player.name in malicious_agents else 5)
    ] for player in players
}

print(players_datetimes)

In [ ]:
import time
import json
import cProfile
import pstats
import pickle
import copy

# Define the function that writes logs for a player
def process_player(player, candidate_agents, agent_config_filename):
    result = []
    
    # Check votes
    ans = check_vote(candidate_agents, player)
    result.append((f"{player.name} votes for {ans}\n", agent_config_filename + "votes_log.txt"))
    
    # Check political scores
    c1 = check_pol(candidate_agents[0], player)
    c2 = check_pol(candidate_agents[1], player)
    result.append((f"{player.name} gives {candidate_agents[0].split()[0]} a score of {c1}\n", agent_config_filename + "pol_log.txt"))
    result.append((f"{player.name} gives {candidate_agents[1].split()[0]} a score of {c2}\n", agent_config_filename + "pol_log.txt"))
    
    return result

def write_logs(results):
    # Write the results to the respective files
    for content, file_name in results:
        with open(file_name, "a") as f:
            f.write(content)


def read_token_data(file_path):
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        return {'prompt_tokens': 0, 'completion_tokens': 0}

episode_length = 240 #961
time_intervals=[]
prompt_token_intervals = []
completion_token_intervals = []
player_copy_list=[]
# with open("../reports/players_all_timesteps.pkl", "wb") as file:
if True:
    start_time = time.time()  # Start timing
    for i in range(episode_length):
    # Parallelize the process using ThreadPoolExecutor
        if i!=0:    
            with concurrent.futures.ThreadPoolExecutor() as executor:
                # Write the episode logs
                    executor.submit(write_logs, [(f"Episode: {i}\n", agent_config_filename + "votes_log.txt")])
                    executor.submit(write_logs, [(f"Episode: {i}\n", agent_config_filename + "pol_log.txt")])            
                    # Process each player in parallel
                    futures = [executor.submit(process_player, p, candidate_agents, agent_config_filename) for p in players]
                    
                    # Write each player's results in parallel
                    for future in concurrent.futures.as_completed(futures):
                        player_results = future.result()
                        executor.submit(write_logs, player_results)
        print(f"Episode: {i}")
        with open("app_logger.txt","a") as a:
            a.write(f"Episode: {i}")

        start_timex = time.time()
        matching_players = []
        # Loop through each player and their associated datetime objects
        for player_name, datetimes in players_datetimes.items():
            added = False  # Flag to check if player was already added
            for datetime_obj in datetimes:
                # Check if the hour and minute match the current time (ignoring seconds and microseconds)
                if datetime_obj.time().hour == clock.now().hour and datetime_obj.time().minute == clock.now().minute:
                    matching_players.append(player_name)
                    added = True
                    break 

            # If player is not added by matching time, check for random addition (15% chance)
            if not added and random.random() < 0.15:
                matching_players.append(player_name)

        print(time.time() - start_timex)

        # Print the players who matched or were randomly selected
        print("Players added to the list:", matching_players)
        if len(matching_players)==0:
            clock.advance()
        else:
            env.step(active_players = matching_players)
            end_timex = time.time()
            with open("time_logger.txt", "a") as f:
                f.write(f"Episode with {len(matching_players)} finished - took {end_timex - start_timex}\n")
    # pickle.dump(players, file)

## Summary and analysis of the episode

In [ ]:
# Summarize the entire story
all_gm_memories = env._memory.retrieve_recent(k=10000, add_time=True)

detailed_story = "\n".join(all_gm_memories)
print("len(detailed_story): ", len(detailed_story))
# print(detailed_story)

episode_summary = model.sample_text(
    f"Sequence of events:\n{detailed_story}"
    "\nNarratively summarize the above temporally ordered "
    "sequence of events. Write it as a news report. Summary:\n",
     max_tokens=3500,
     terminators=(),
)
print(episode_summary)

In [ ]:
# Summarise the perspective of each player
player_logs = []
player_log_names = []
for player in players:
  name = player.name
  detailed_story = "\n".join(memories[player.name].retrieve_recent(
      k=1000, add_time=True))
  summary = ""
  summary = model.sample_text(
      f"Sequence of events that happened to {name}:\n{detailed_story}"
      "\nWrite a short story that summarises these events.\n",
       max_tokens=3500,
       terminators=()
)

  all_player_mem = memories[player.name].retrieve_recent(k=1000, add_time=True)
  all_player_mem = ["Summary:", summary, "Memories:", *all_player_mem]
  player_html = html_lib.PythonObjectToHTMLConverter(all_player_mem).convert()
  player_logs.append(player_html)
  player_log_names.append(f"{name}")



## Build and display HTML log of the experiment

In [ ]:
history_sources = [env, direct_effect_externality]
histories_html = [html_lib.PythonObjectToHTMLConverter(history.get_history()).convert() for history in history_sources]
histories_names = [history.name for history in history_sources]

In [ ]:
gm_mem_html = html_lib.PythonObjectToHTMLConverter(all_gm_memories).convert()

tabbed_html = html_lib.combine_html_pages(
    [*histories_html, gm_mem_html, *player_logs],
    [*histories_names, "GM", *player_log_names],
    summary=episode_summary,
    title="Mastodon experiment",
)

tabbed_html = html_lib.finalise_html(tabbed_html)
with open("index5-55.html","w",encoding="utf-8") as f:
    f.write(tabbed_html)


In [ ]:
display.HTML(tabbed_html)

## Interact with a specific player

In [ ]:
sim_to_interact = "Alice"
user_identity = "a close friend"
interaction_premise = f"{sim_to_interact} is talking to {user_identity}\n"

player_names = [player.name for player in players]
player_by_name = {player.name: player for player in players}
selected_player = player_by_name[sim_to_interact]
interrogation = interaction_premise

In [ ]:
utterance_from_user = (
    "Hey Alice, did you post anything on Mastodon today?"
)

interrogation += f"{user_identity}: {utterance_from_user}"
player_says = selected_player.say(interrogation)
interrogation += f"\n{sim_to_interact}: {player_says}\n"
print(interrogation)

## Check timeline

Finally, check the full public timeline for the Mastodon server.

You may also check this directly at https://social-sandbox.com (you'll need to log in as a user).

In [ ]:
timeline = get_public_timeline(limit=None)
print_timeline(timeline)